In [19]:
import numpy as np
import pandas as pd
import datetime as dt
from gurobipy import *
from BOM_graph.StudyBOM import GenerateGraph
from Environments.chargeEnvironment import chargeEnv
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# Definicion de los parametros del entorno gurobi

#Dado el número de variables de este modelo, es necesario utilizar gurobi con una licencia académica (se obtiene gratuitamente en la web de gurobi)
params = {"WLSACCESSID" : '5cbfde7e-b7bd-44e3-9e2d-fb335f5a2deb', "WLSSECRET" : 'f526471f-ed2c-462d-a933-1241228cd704', "LICENSEID" : 2489204 }
env = Env(params = params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2489204
Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es


In [13]:
# Definición de los parámetros del problema
BOM, MixedItems, PurchaseItems, RouteItems, Orders, Stock, Tenv = chargeEnv(mode = "default")

# Grafo
G = GenerateGraph(BOM, typeG_ND = False, connected = True)

# Items
NN = sorted(G.nodes)
NN_len = sorted(NN)
K1 = sorted(RouteItems["MyBOMITEMID"].unique().tolist())
K2 = sorted(PurchaseItems["MyBOMITEMID"].unique().tolist())
K3 = sorted(MixedItems["MyBOMITEMID"].unique().tolist())
LEVEL0 = sorted(BOM.loc[ BOM["LEVEL"] == 0,"MyBOMITEMID"].unique().tolist())

# Arcos 
arcos = sorted(G.edges)

# Layers
layers_dict = {}
for node in G.nodes:
    layer = G.nodes[node]["layer"]
    if layer not in layers_dict:
        layers_dict[layer] = []
    layers_dict[layer].append(node)
layers = [sorted(nodes) for layer, nodes in sorted(layers_dict.items())]

# Conjuntos N(i) e inversos
N = {}
# Encontrar los ítems necesarios para cada ítem i en Kt ∪ K3
for i in sorted(K1+K3):
    N[i] = [j for _, j in G.edges(i)]

N_reverse = {j: [] for j in NN}
for j, neighbors in N.items():
    for i in neighbors:
        N_reverse[i].append(j)

# Clientes
R = sorted(Orders["CUSTOMERID"].unique().tolist())
R_len = len(R)

# Periodos de tiempo en el horizonte temporal
date_range = pd.date_range(start= dt.datetime.now().strftime("%Y-%m-01"), periods= Tenv, freq='MS')
T = date_range.tolist() # De hoy a 12 meses

# Demanda y precios de venta
# LEVEL0 = layers[0]+[39]
# Map MyBOMITEMID and CUSTOMERID to indices
item_indices = {item: idx for idx, item in enumerate(LEVEL0)} # El 39 es un ítem a nivel 0 y Layer 1
customer_indices = {customer: idx for idx, customer in enumerate(R)}
D = []
B = []
for period_start in T:
    period_end = period_start + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # End of the month
    period_df = Orders[(Orders['END_DATE'] >= period_start) & (Orders['END_DATE'] <= period_end)]
    
    # Create matrices for quantities and prices
    period_matrixD = np.zeros((len(LEVEL0), R_len))
    period_matrixB = np.zeros((len(LEVEL0), R_len))
    
    # Fill the matrices
    for _, row in period_df.iterrows():
        item_idx = item_indices[row['MyBOMITEMID']]
        customer_idx = customer_indices[row['CUSTOMERID']]
        period_matrixD[item_idx, customer_idx] = row['QUANTITY']
        period_matrixB[item_idx, customer_idx] = row['UNITPRICE_EUR']
    
    # Append the period matrices to the lists
    D.append(period_matrixD)
    B.append(period_matrixB)
    
# Costes 
c1 = {**dict(zip(RouteItems["MyBOMITEMID"], RouteItems["RUNTIME_COST"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["RUNTIME_COST"]))}
c2 = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["UNITPRICE_Compra"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["UNITPRICE_Compra"]))}

# MOQs
MOQ1 = {**dict(zip(RouteItems["MyBOMITEMID"], RouteItems["MOQ_Fabricacion"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["MOQ_Fabricacion"]))}
MOQ2 = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["MOQ_Compra"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["MOQ_Compra"]))}

# Lead times
lt = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["LEADTIME"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["LEADTIME"]))}

# Stock
I_0 = dict(zip(Stock["MyBOMITEMID"], Stock["STOCK"]))


# Matriz alpha
# Construir la matriz alpha
alpha = {}

for i in sorted(K1 + K3):
    alpha[i] = {}
    for j in N[i]:
        maxibo_qty = BOM[(BOM['MyPARENTBOMITEMID'] == i) & (BOM['MyBOMITEMID'] == j)]['MAXIBOQTY']
        if not maxibo_qty.empty:
            alpha[i][j] = maxibo_qty.values[0]
        else:
            alpha[i][j] = 0

In [14]:
# Inicialización del modelo
modelo = Model("Ejercicio", env = env)

In [15]:
# Definición de las variables
indices_x = [(i,t) for t in range(1,len(T)) for i in K1+K3]
indices_z1 = [(i,t) for t in range(1,len(T))  for i in K1+K3]
indices_y = [(i,t) for t in range(1,len(T))  for i in K2+K3]
indices_z2 = [(i,t) for t in range(1,len(T))  for i in K2+K3]
indices_w = [(i, r, t) for t in range(1,len(T))  for i in LEVEL0 for r in R]
indices_I = [(i,t) for t in range(1,len(T))  for i in NN]

x = modelo.addVars(indices_x, lb = 0,vtype = GRB.INTEGER, name = "x") 
z1 = modelo.addVars(indices_z1, lb = 0,vtype = GRB.BINARY, name = "z1")
y = modelo.addVars(indices_y, lb = 0, vtype = GRB.INTEGER, name = "y")
z2 = modelo.addVars(indices_z2, lb = 0,vtype = GRB.BINARY, name = "z2")
w = modelo.addVars(indices_w, lb = 0, vtype= GRB.BINARY, name = "w") 
It =  modelo.addVars(indices_I, lb = 0,vtype= GRB.INTEGER, name = "It")

modelo.update()

In [16]:
# Inventario para el primer periodo, para items a nivel 0
r10 = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] - quicksum(D[1][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R) for i in list(set(K1).intersection(set(LEVEL0)))
    ),name="R10" )
for i in set(K2).intersection(set(LEVEL0)):
    if lt[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + y[i, 1-lt[i]] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R20_{i}"
        )
    if lt[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R30_{i}"
        )   
for i in set(K3).intersection(set(LEVEL0)):
    if lt[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + x[i, 1] + y[i, 1-lt[i]] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R40_{i}"
        )
    if lt[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + x[i, 1] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R50_{i}"
        )
    

# Inventario para el primer periodo, para items a otro nivel
r1a = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] - quicksum(alpha[j][i]*x[j,1] for j in N_reverse[i]) for i in list(set(K1).intersection(set().union(*layers[1:])))
    ),name="R1a" )
for i in set(K2).intersection(set().union(*layers[1:])):
    if lt[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + y[i, 1-lt[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R2a_{i}"
        )
    if lt[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R3a_{i}"
        )
for i in set(K3).intersection(set().union(*layers[1:])):
    if lt[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + x[i, 1] + y[i, 1-lt[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R4a_{i}"
        )
    if lt[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + x[i, 1] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R5a_{i}"
        )
    

# Inventario para el resto de periodos, para items a nivel 0
r10t = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t] - quicksum(D[t][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R) for i in list(set(K1).intersection(set(LEVEL0))) for t in range(2, len(T))
    ),name="R10t" )
for i in set(K2).intersection(set(LEVEL0)):
    for t in range(2, len(T)):
        if lt[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + y[i, t-lt[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R20t_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R30t_{i}_{t}"
            )
for i in set(K3).intersection(set(LEVEL0)):
    for t in range(2, len(T)):
        if lt[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + x[i, t] + y[i, t-lt[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R40t_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + x[i, t] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R50t_{i}_{t}"
            )

# Inventario para el resto de periodos, para items a otro nivel
r1at = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t] - quicksum(alpha[j][i]*x[j, t] for j in N_reverse[i]) for i in list(set(K1).intersection(set().union(*layers[1:]))) for t in range(2, len(T))
    ),name="R1at" )
for i in set(K2).intersection(set().union(*layers[1:])):
    for t in range(2, len(T)):
        if lt[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + y[i, t-lt[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R2a_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R3a_{i}_{t}"
            )
for i in set(K3).intersection(set().union(*layers[1:])):
    for t in range(2, len(T)):
        if lt[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + x[i, t] + y[i, t-lt[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R4a_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + x[i, t] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R5a_{i}_{t}"
            )


# Restricciones de MOQs

r6 = modelo.addConstrs(
    (x[i,t] >= z1[i,t]*MOQ1[i] for i in K1+K3 for t in range(1, len(T))
     ),name="R6" )
r7 = modelo.addConstrs(
    (y[i,t] >= z2[i,t]*MOQ2[i] for i in K2+K3 for t in range(1, len(T))
     ),name="R7" )

# Restricciones de activacion de variables binarias
r8 = modelo.addConstrs(
    (x[i,t] <= z1[i,t]*420000 for i in K1+K3 for t in range(1, len(T))
     ),name="R8" )
r9 = modelo.addConstrs(
    (y[i,t] <= z2[i,t]*420000 for i in K2+K3 for t in range(1, len(T))
     ),name="R9" )

modelo.update()

In [17]:
# Definición de la función objetivo
modelo.setObjective(quicksum(quicksum(D[t][item_indices[i],customer_indices[r]]*B[t][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R for i in LEVEL0) 
                             - quicksum(c1[i]*x[i,t] for i in K1+K3)
                             - quicksum(c2[i]*y[i,t] for i in K2+K3)
                             for t in range(1, len(T))), sense = GRB.MAXIMIZE)

In [18]:
# Optimizacion
modelo.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es
Optimize a model with 3840 rows, 37524 columns and 11034 nonzeros
Model fingerprint: 0x03bd8fa2
Variable types: 0 continuous, 37524 integer (35148 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [9e-02, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+04]
Presolve removed 2158 rows and 35738 columns
Presolve time: 0.03s
Presolved: 1682 rows, 1786 columns, 5167 nonzeros
Variable types: 0 continuous, 1786 integer (793 binary)

Root relaxation: objective 4.807296e+06, 655 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [21]:
# for t in range(1,len(T)+1):
#     for i in K1+K3:
#         if z1[i,t].X!=0:
#             print(i,t)
#             print(x[i,t].X)
            
# for t in range(1,len(T)+1):
#     for i in K2+K3:
#         if z2[i,t].X!=0:
#             print(i,t)
#             print(y[i,t].X)

for t in range(1, len(T)):
    for i in LEVEL0:
        for r in R:
            if w[i, r, t].X != 0 and D[t][item_indices[i],customer_indices[r]] != 0:
                print(f"Item, cliente, horizonte temporal: {i, r, t}")
                print(f"Demanda {D[t][item_indices[i],customer_indices[r]]}")
                if t == 1:
                    print(f"Stock antes: {I_0[i]}")
                else:
                    print(f"Stock antes: {It[i,t-1].X}")
                print(f"Stock despues: {It[i,t].X}")
                if i in K1+K3:
                    print(f"Cantidad Producida {x[i,t].X}")
                if i in K2+K3:
                    print(f"Cantidad Comprada {y[i,t].X}")
                    if lt[i]<t:
                        print(f"Cantidad comprada previamente que llega ahora {y[i,t-lt[i]].X}")
                    if i in K2:
                        lead_time = PurchaseItems.loc[PurchaseItems["MyBOMITEMID"] == i, "LEADTIME"].values[0]
                    if i in K3:
                        lead_time = MixedItems.loc[MixedItems["MyBOMITEMID"] == i, "LEADTIME"].values[0]
                    print(f"Lead time {lead_time}")
            else:
                if D[t][item_indices[i],customer_indices[r]] != 0:
                    print(f"Item, cliente, horizonte temporal: {i, r, t}")
                    print(f"NO SE SATISFACE")
    print("------------------------------------------")

                        

------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
Item, cliente, horizonte temporal: (3, '101695', 7)
Demanda 23.0
Stock antes: 997.0
Stock despues: 1042.0
Cantidad Producida 68.0
Cantidad Comprada 0.0
Cantidad comprada previamente que llega ahora -0.0
Lead time 6
Item, cliente, horizonte temporal: (23, '101371', 7)
Demanda 270.0
Stock antes: 2183.0
Stock despues: 1638.0
Cantidad Producida -0.0
Cantidad Comprada -0.0
Cantidad comprada previamente que llega ahora -0.0
Lead time 0
Item, cliente, horizonte temporal: (23, '101706', 7)
Demanda 275.0
Stock antes: 2183.0
Stock despues: 1638.0
Cantidad Producida -0.0
Cantidad Comprada -0.0
Cantidad comprada previamente que llega ahora -0.0
Lead time 0
Item, cliente, horizonte temporal: (36, '777774', 7)
Demanda 1133.0
Stock antes: 29.0
Sto

In [ ]:
for t in range(1,len(T)):
    for i in K2+K3:
        if z2[i,t].X!=0:
            print(f"Item, horizonte temporal {i, t}")
            print(f"Cantidad comprada {y[i,t].X}")
            print(f"Lead Time {lt[i]}")
            print(f"MOQ {MOQ2[i]}")
print("------------------------------------------")

In [ ]:
for t in range(1,len(T)):
    for i in K1+K3:
        if z1[i,t].X!=0:
            print(f"Item, horizonte temporal {i, t}")
            print(f"Cantidad producida {x[i,t].X}")
            print(f"MOQ {MOQ1[i]}")
        else:
            if x[i,t].X != 0:
                print(f"No debería producir {i, t}")
                print(f"Cantidad producida {x[i,t].X}")

In [ ]:
NoSatisfecha = 0
totalPedidos = 0
udsNoSatisfecha = 0
totalUds = 0
for t in range(1, len(T)):
    for i in LEVEL0:
        for r in R:
            if D[t][item_indices[i],customer_indices[r]] != 0:
                if w[i, r, t].X == 0:
                    NoSatisfecha += 1
                    udsNoSatisfecha += D[t][item_indices[i],customer_indices[r]]
                totalPedidos += 1
                totalUds += D[t][item_indices[i],customer_indices[r]]


print(udsNoSatisfecha/totalUds*100)
print(NoSatisfecha/totalPedidos*100)
print(modelo.getAttr("ObjVal"))


In [ ]:
modelo.close()
env.close()